#### <span style="color:#grey"> __Formation South Green 2022 - Structural Variants Detection by using short and long reads__ </span>

# <span style="color:#006E7F">  <center> __DAY 4 : How to identify large variants using whole-genome assemblies ?__ </center> </span>

Created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)

***

# <span style="color: #006E7F">Table of contents</span>
<a class="anchor" id="home"></a>

[I - using nucmer,assemblytics](#SVassemblytics)

* [Run minimap2](#minimap)
* [Run syri](#syri)

[II - using minimap2, syri](#SVsyri)

* [Run minimap2](#minimap)
* [Run syri](#syri)

* Comparison

</span>

***



## <span style="color:#006E7F">__0 - Preparing working environment__ <a class="anchor" id="work"></a></span>  

### <span style="color: #4CACBC;"> Create the working directory /work/day4-SV</span>  

In [12]:
mkdir -p /home/jovyan/work/day4-WGA-SV
cd /home/jovyan/work/day4-WGA-SV
ls

### <span style="color: #4CACBC;"> Download two genomes</span>  

In [13]:
wget https://itrop.ird.fr/sv-training/Assemblies.tar.gz
tar xvf Assemblies.tar.gz && rm Assemblies.tar.gz

--2022-06-17 12:48:25--  https://itrop.ird.fr/sv-training/Assemblies.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9150274 (8.7M) [application/x-gzip]
Saving to: ‘Assemblies.tar.gz’

Assemblies.tar.gz   100%[===================>]   8.73M  37.3MB/s    in 0.2s    

2022-06-17 12:48:25 (37.3 MB/s) - ‘Assemblies.tar.gz’ saved [9150274/9150274]

Assemblies/
Assemblies/assembly-stats.txt
Assemblies/A8_assembly.fasta
Assemblies/5417_assembly.fasta


In [14]:
ls -R

.:
Assemblies

./Assemblies:
5417_assembly.fasta  A8_assembly.fasta  assembly-stats.txt


## <span style="color:#006E7F">__I - Using nucmer, assemblytics__ <a class="anchor" id="SVassemblytics"></a></span>  

### <span style="color: #4CACBC;"> Initialization of two variables "genome"</span>  


In [1]:
dir_genome="/home/jovyan/work/day4-WGA-SV/"
reference_assembly=$dir_genome"Assemblies/A8_assembly.fasta"
query_assembly=$dir_genome"Assemblies/5417_assembly.fasta"

### <span style="color: #4CACBC;"> Create the working directory for SYRI analysis</span>  

In [2]:
dir_SV=$dir_genome"SYRI/"
mkdir -p $dir_SV
cd $dir_SV

### <span style="color: #4CACBC;"> Aligning genomes using `Nucmer` <a class="anchor" id="nucmer"></a></span>  

__Alignement of the two genomes__

In [42]:
nucmer --maxmatch $reference_assembly $query_assembly

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "out.ntref" of length 15306200
# construct suffix tree for sequence of length 15306200
# (maximum reference length is 2305843009213693948)
# (maximum query length is 18446744073709551615)
# process 153062 characters per dot
#...................................................................................................
# CONSTRUCTIONTIME /opt/conda/opt/mummer-3.23/mummer out.ntref 12.36
# reading input file "/home/jovyan/work/day4-WGA-SV/Assemblies/5417_assembly.fasta" of length 15276944
# matching query-file "/home/jovyan/work/day4-WGA-SV/Assemblies/5417_assembly.fasta"
# against subject-file "out.ntref"
# COMPLETETIME /opt/conda/opt/mummer-3.23/mummer out.ntref 48.16
# SPACE /opt/conda/opt/mummer-3.23/mummer out.ntref 29.56
4: FINISHING DATA


In [43]:
ls -lrt

total 1788
-rw-r--r-- 1 jovyan users 1829452 Jun 17 13:08 out.delta


__Filtering nucmer results__

Remove small and lower quality alignments


In [44]:
delta-filter -m -i 90 -l 100 out.delta > out.filtered.delta

In [48]:
ls -lrt

total 2540
-rw-r--r-- 1 jovyan users 1829452 Jun 17 13:08 out.delta
-rw-r--r-- 1 jovyan users  667061 Jun 17 13:08 out.filtered.delta
-rw-r--r-- 1 jovyan users  100931 Jun 17 13:08 out.filtered.coords


In [45]:
show-coords -THrd out.filtered.delta > out.filtered.coords

In [49]:
ls -lrt

total 2540
-rw-r--r-- 1 jovyan users 1829452 Jun 17 13:08 out.delta
-rw-r--r-- 1 jovyan users  667061 Jun 17 13:08 out.filtered.delta
-rw-r--r-- 1 jovyan users  100931 Jun 17 13:08 out.filtered.coords


### <span style="color: #4CACBC;"> SV calling using syri <a class="anchor" id="siri"></a></span>  

In [4]:
syri -c out.filtered.coords -d out.filtered.delta -r $reference_assembly -q $query_assembly

Finished syri


In [14]:
ls

out.delta            plotsr.pdf      syri.log      syri.vcf
out.filtered.coords  plotsr_pos.txt  syri.out      tracks.txt
out.filtered.delta   SNPs.bed        syri.summary


In [15]:
cat syri.summary

#Structural annotations
#Variation_type	Count	Length_ref	Length_qry
Syntenic regions	197	14079055	14107736
Inversions	32	397553	259052
Translocations	93	91312	90081
Duplications (reference)	152	79812	-
Duplications (query)	161	-	83493
Not aligned (reference)	372	667759	-
Not aligned (query)	413	-	738403


#Sequence annotations
#Variation_type	Count	Length_ref	Length_qry
SNPs	239492	239492	239492
Insertions	31092	-	114022
Deletions	38036	119894	-
Copygains	32	-	13466
Copylosses	28	17778	-
Highly diverged	707	487347	383289
Tandem repeats	23	8965	10360


In [17]:
head syri.vcf

##fileformat=VCFv4.3
##fileDate=20220617
##source=syri
##ALT=<ID=SYN,Description="Syntenic region">
##ALT=<ID=INV,Description="Inversion">
##ALT=<ID=TRANS,Description="Translocation">
##ALT=<ID=INVTR,Description="Inverted Translocation">
##ALT=<ID=DUP,Description="Duplication">
##ALT=<ID=INVDP,Description="Inverted Duplication">
##ALT=<ID=SYNAL,Description="Syntenic alignment">


### <span style="color: #4CACBC;"> Extracting all SNP from syri  <a class="anchor" id="siri"></a></span>  

In [7]:
cat syri.out | grep SNP | cut -f 1,2 > a
cat syri.out | grep SNP | cut -f 4,5 > b
cat syri.out | grep SNP | cut -f 3 > c
awk '{ print $1+1 }' c > d
paste a d c > SNPs.bed
rm a b c d


In [8]:
echo -e $reference_assembly'\t ref' >> plotsr_pos.txt
echo -e $query_assembly'\t query' >> plotsr_pos.txt


In [13]:
plotsr --sr syri.out --genomes plotsr_pos.txt -s 500 -o plotsr.pdf -H 8 -W 5

Plotsr - WARNING - Plot plotsr.pdf generated.
Finished
